<a href="https://colab.research.google.com/github/NihaarikaAgarwal/Movie-Recommendation/blob/main/Movie_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# Install the Surprise library for collaborative filtering
!pip install scikit-surprise

In [17]:
from google.colab import files
uploaded = files.upload()

Saving u.item to u.item
Saving u.data to u (1).data


In [18]:
import pandas as pd

# Load the dataset
column_names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('u.data', sep='\t', names=column_names)

# Drop the timestamp column as it's not needed
df = df.drop('timestamp', axis=1)

# Display the first few rows
df.head()


,user_id,item_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


In [19]:
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import SVD

# Load the dataset into Surprise format
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['user_id', 'item_id', 'rating']], reader)

# Split the data into train and test sets
trainset, testset = train_test_split(data, test_size=0.2)

# Use Singular Value Decomposition (SVD) for recommendations
model = SVD()

# Train the model on the training set
model.fit(trainset)

# Evaluate the model on the test set
from surprise import accuracy

predictions = model.test(testset)
print("RMSE:", accuracy.rmse(predictions))

RMSE: 0.9431
RMSE: 0.9430708409872157


In [20]:
# Get all unique movie IDs
all_movies = df['item_id'].unique()

# Function to recommend top-N movies for a given user
def recommend_movies(user_id, n=5):
    # Get movies the user has already rated
    rated_movies = df[df['user_id'] == user_id]['item_id']

    # Predict ratings for movies the user has not rated
    unrated_movies = [movie for movie in all_movies if movie not in rated_movies]
    predictions = [model.predict(user_id, movie) for movie in unrated_movies]

    # Sort by predicted rating
    recommendations = sorted(predictions, key=lambda x: x.est, reverse=True)[:n]

    # Return movie IDs and predicted ratings
    return [(pred.iid, pred.est) for pred in recommendations]

# Example: Recommend top 5 movies for user 1
user_id = 1
recommended_movies = recommend_movies(user_id, n=5)
print("Top 5 Recommended Movies for User", user_id)
print(recommended_movies)


Top 5 Recommended Movies for User 1
[(100, 5), (172, 4.90539582869777), (50, 4.86151510353973), (48, 4.8452888260407505), (483, 4.824891558445141)]


In [21]:
# Load movie titles
movie_titles = pd.read_csv('u.item', sep='|', encoding='latin-1', header=None, usecols=[0, 1], names=['item_id', 'title'])

# Create a mapping from movie ID to title
id_to_title = dict(zip(movie_titles['item_id'], movie_titles['title']))

# Display recommendations with titles
for movie_id, rating in recommended_movies:
    print(f"Movie: {id_to_title[int(movie_id)]}, Predicted Rating: {rating:.2f}")


Movie: Fargo (1996), Predicted Rating: 5.00
Movie: Empire Strikes Back, The (1980), Predicted Rating: 4.91
Movie: Star Wars (1977), Predicted Rating: 4.86
Movie: Hoop Dreams (1994), Predicted Rating: 4.85
Movie: Casablanca (1942), Predicted Rating: 4.82
